In [ ]:
import torchvision
import torchvision.transforms as transforms
import numpy as np
from PIL import Image
import random
import torch
import os
import shutil
import matplotlib.pyplot as plt
PATH_1 = "./data/dataset/LT_with_noise/LT_noise/combined_noise_nl_0.2/"
PATH_2 = "./data/dataset/LT_with_noise/LT_noise/combined_noise_nl_0.4/"
PATH_IN = "./data/dataset/LT_with_noise/imagenet_noise/0.0_red"
PATH_OUT = "./data/dataset/LT_with_noise/noise_pool/"


In [ ]:

def get_img_num_per_cls(raw_len, cls_num, imb_type, imb_factor):
    img_max = raw_len
    img_num_per_cls = []
    if imb_type == 'exp':
        for cls_idx in range(cls_num):
            num = img_max * (imb_factor**(cls_idx / (cls_num - 1.0)))
            img_num_per_cls.append(int(num))
    elif imb_type == 'step':
        for cls_idx in range(cls_num // 2):
            img_num_per_cls.append(int(img_max))
        for cls_idx in range(cls_num // 2):
            img_num_per_cls.append(int(img_max * imb_factor))
    else:
        img_num_per_cls.extend([int(img_max)] * cls_num)
    return img_num_per_cls

In [ ]:
noise_rate = 0
cls_num = 100
imb_type = 'exp'
imbalance_ratio = 0.05
raw_len = 1000
img_num_list = get_img_num_per_cls(raw_len, cls_num, imb_type, imbalance_ratio)

In [ ]:
i = 0

for cls in range(0,cls_num):
    cls = str(cls)
    if len(cls) ==1:
      cls = '0'+cls
    os.mkdir(os.path.join(PATH_OUT,cls))
    num = img_num_list[i]
    i = i+1
    sub_path = os.path.join(PATH_IN,cls)
    name_list = os.listdir(sub_path)
    sample_num = int(num*(1-noise_rate))
    noise_num = int(num*noise_rate)
    sample_list,noise_list = [],[]
    for pic_name in name_list:
        #if pic_name[0] == 'n':
        #    sample_list.append(pic_name)
        #else:
        #    noise_list.append(pic_name)
        sample_list.append(pic_name)

    random.shuffle(sample_list)
    sample_list = sample_list[:sample_num]

    for sample in sample_list:
        in_path = os.path.join(sub_path,sample)
        out_path = os.path.join(PATH_OUT,cls)
        shutil.copy(in_path,out_path)
    
    '''
    random.shuffle(noise_list)
    noise_list = noise_list[:noise_num]


    for noise in noise_list:
        in_path = os.path.join(sub_path,noise)
        out_path = os.path.join(PATH_OUT,cls)
        shutil.copy(in_path,out_path)
    '''

In [ ]:

noise_rate = 0.2
# create noise pool
for cls in os.listdir(PATH_2):
    temp_path = os.path.join(PATH_2,cls)
    sample_list = os.listdir(temp_path)
    true_sample_list =[]
    for sample in sample_list:
        if sample[0] == 'n' :
            true_sample_list.append(sample)

    sample_num = int(noise_rate * 1000)
    random.shuffle(true_sample_list)
    true_sample_list = true_sample_list[:sample_num]
    for sample in true_sample_list:
        shutil.move(os.path.join(temp_path,sample),PATH_OUT)
    

In [ ]:

print(true_sample_list)

In [ ]:
# add noise

noise_rate = 0.2
for cls in os.listdir(PATH_2):
    temp_path = os.path.join(PATH_2,cls)
    sample_list = os.listdir(PATH_OUT)
    sample_num = int(noise_rate * 1000)
    random.shuffle(sample_list)
    sample_list = sample_list[:sample_num]
    for sample in sample_list:
        shutil.move(os.path.join(PATH_OUT,sample),temp_path)
